In [12]:

%%javascript

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-k', {
    help : 'move up selected cells',
    help_index : 'jupyter-notebook:move-selection-up',
    handler : function (event) {
        IPython.notebook.move_selection_up();
        return false;
    }}
);

IPython.keyboard_manager.command_shortcuts.add_shortcut('Ctrl-j', {
    help : 'move down selected cells',
    help_index : 'jupyter-notebook:move-selection-down',
    handler :  function (event) {
        IPython.notebook.move_selection_down();
        return false;
    }}
);

<IPython.core.display.Javascript object>

In [154]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import inline as inline

%matplotlib inline

pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 23)
pd.set_option('display.max_rows', 100)

pd.options.mode.chained_assignment = None

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

df = pd.read_csv('events.csv', low_memory = False)

df.describe()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,1011288,1011288,1011288,82756,563838,564284,563836,563836,563836,221699,48967,3598,82796,50957,87378,87378,87378,87378,87378,87378,87378,87378,87378
unique,793805,11,27624,227,3574,202,5,8,63,35310,5851,14,24,4,7,2,1939,93,46,4,282,121,343
top,2018-05-15 15:56:06,viewed product,71492f2b,/,2830.0,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",Iphone,CustomerService,google,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,11,528931,2771,28323,4282,50916,243014,190833,132960,594,1207,1528,58153,50240,44193,60480,15819,24996,84308,44239,30009,19675,22611


In [66]:
df_campaign = df[df.campaign_source.notnull()][['person', 'campaign_source', 'timestamp']].drop_duplicates('person')
df_campaign.describe()

,person,campaign_source,timestamp
count,21306,21306,21306
unique,21306,22,21216
top,897da1dd,google,2018-05-15 15:56:06
freq,1,16699,3


In [151]:
df_campaign_users = df.loc[df['person'].isin(df_campaign['person']), :]
df_campaign_buys = df_campaign_users.loc[df_campaign_users['event'] == 'conversion']
df_campaign_users_buys = df_campaign_buys.drop_duplicates('person')

df_campaign_users_buys.describe()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,559,559,559,0.0,559,559,559,559,559,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
unique,559,1,559,0.0,358,104,4,7,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
top,2018-04-29 13:18:15,conversion,69ed21aa,NaN,6371.0,iPhone 5s,Bom,16GB,Preto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,559,1,NaN,12,41,234,271,183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Top Campañas

In [27]:
df_campaign['campaign_source'].value_counts()

google              16699
criteo               2831
rtbhouse              371
FacebookAds           292
zanox                 254
voxus                 212
emblue                193
bing                  159
afilio                130
buscape                39
indexa                 26
Facebook               20
rakuten                19
afiliado               16
mercadopago            13
datacrush              10
blog                    9
manifest                4
yotpo                   4
FacebookSocial          3
MARKETING SOCIAL        1
socialmedia             1
Name: campaign_source, dtype: int64

In [69]:
campaign_conversion = df_campaign.loc[df_campaign['person'].isin(df_campaign_buys['person'])]
campaign_conversion = (campaign_conversion['campaign_source'].value_counts()).to_frame()
campaign_conversion.columns = ['conversion']

df_not_in_conversion_campaign = df_campaign.loc[~df_campaign['person'].isin(df_campaign_buys['person'])]
campaign_no_conversion = df_not_in_conversion_campaign['campaign_source'].value_counts().to_frame()
campaign_no_conversion.columns = ['no conversion']

In [71]:
table_campaign = campaign_conversion.join(campaign_no_conversion, how='left', lsuffix ='conversion', rsuffix ='no conversion')

table_campaign["total"] = table_campaign['conversion'] + table_campaign['no conversion']
table_campaign['porcentile'] = (table_campaign['conversion']/table_campaign["total"])*100
table_campaign

,conversion,no conversion,total,porcentile
google,403,16296,16699,2.413318
criteo,47,2784,2831,1.660191
rtbhouse,32,339,371,8.625337
zanox,21,233,254,8.267717
FacebookAds,15,277,292,5.136986
emblue,9,184,193,4.663212
bing,9,150,159,5.660377
afilio,8,122,130,6.153846
buscape,7,32,39,17.948718
voxus,2,210,212,0.943396


In [86]:
df_campaign['timestamp'] = pd.to_datetime(df_campaign['timestamp'])
df_campaign['month'] = df_campaign['timestamp'].dt.month
df_campaign['count'] = 1
df_campaign_month = df_campaign.groupby('month').agg({'count':'sum'})
df_campaign_month.columns = ['people of campaign']

In [87]:
df_campaign_users_buys['timestamp'] = pd.to_datetime(df_campaign_users_buys['timestamp'])
df_campaign_users_buys['month'] = df_campaign_users_buys['timestamp'].dt.month
df_campaign_users_buys['count'] = 1
df_conversion_month = df_campaign_users_buys.groupby('month').agg({'count':'sum'})
df_conversion_month.columns = ['conversions']


In [88]:
month_conversions_people = df_conversion_month.join(df_campaign_month, how='left', lsuffix ='conversion', rsuffix ='people of campaign')
month_conversions_people

,conversions,people of campaign
month,,
1,45,392
2,44,416
3,150,942
4,152,1312
5,152,10041
6,16,8203


## Graph cantidad de entries de campaña

In [93]:
df_test = df_campaign.loc[df_campaign['month'] == 1]
df_test = df_test.loc[df_test['person'].isin(df_campaign_users_buys['person'])]
df_test.campaign_source.value_counts()

google         46
criteo          6
zanox           3
FacebookAds     3
bing            2
indexa          1
buscape         1
afilio          1
Name: campaign_source, dtype: int64

In [94]:
df_test = df_campaign.loc[df_campaign['month'] == 2]
df_test = df_test.loc[df_test['person'].isin(df_campaign_users_buys['person'])]
df_test.campaign_source.value_counts()

google         55
rtbhouse        5
criteo          3
emblue          2
afilio          2
buscape         1
FacebookAds     1
Name: campaign_source, dtype: int64

In [95]:
df_test = df_campaign.loc[df_campaign['month'] == 3]
df_test = df_test.loc[df_test['person'].isin(df_campaign_users_buys['person'])]
df_test.campaign_source.value_counts()

google         107
FacebookAds     10
zanox            7
criteo           6
rtbhouse         5
buscape          5
afilio           3
emblue           2
bing             2
Facebook         1
indexa           1
Name: campaign_source, dtype: int64

In [96]:
df_test = df_campaign.loc[df_campaign['month'] == 4]
df_test = df_test.loc[df_test['person'].isin(df_campaign_users_buys['person'])]
df_test.campaign_source.value_counts()

google         96
criteo         18
rtbhouse        7
emblue          4
zanox           4
blog            2
bing            1
FacebookAds     1
Name: campaign_source, dtype: int64

In [97]:
df_test = df_campaign.loc[df_campaign['month'] == 5]
df_test = df_test.loc[df_test['person'].isin(df_campaign_users_buys['person'])]
df_test.campaign_source.value_counts()

google      87
criteo      13
rtbhouse    12
zanox        4
bing         3
voxus        2
afilio       2
emblue       1
Name: campaign_source, dtype: int64